# AprilTag API Usage demo

This Jupyter Notebook provides a complete guide to AprilTag recognition using computer vision techniques. It covers how to print an AprilTag, perform camera calibration with `OpenCV`, and detect tags in captured images. The notebook demonstrates how to retrieve each tag’s ID as well as its relative position and orientation using the `pupil-apriltags` library, enabling accurate spatial understanding from visual input.

## Requirements

This notebook uses the following Python libraries:

- `pupil-apriltags` — for AprilTag detection and pose estimation
- `opencv-python` — for camera calibration and image processing
- `numpy`, `matplotlib` — for math and visualization

In [ ]:
import importlib
import sys

for pkg in ["pupil-apriltags", "opencv-python", "numpy", "matplotlib"]:
    try:
        importlib.import_module(pkg if pkg != "opencv-python" else "cv2")
    except ImportError:
        print(f"Installing {pkg} ...")
        get_ipython().run_line_magic('pip', f'install {pkg}')


## Preparation

### Print AprilTag images

You can find the images for the tags in the [apriltag-imgs repo](https://github.com/AprilRobotics/apriltag-imgs). For the vast majority of applications, the tagStandard41h12 family will be the correct choice. Scale up the images in your favorite editor and print them out or use the [helper function below](#get-printable-apriltags) to automatically generate a printable SVG version of the tag at your desired size.

Some heuristics for when to choose other tag families:

1. If you need more tags, use tagStandard52h13
2. If you need to maximize the use of space on a small circular object, use tagCircle49h12 (or tagCircle21h7).
3. If you want to make a recursive tag use tagCustom48h12.
4. If you want compatibility with the ArUcO detector use tag36h11

If none of these fit your needs, generate your own custom tag family [here](https://github.com/AprilRobotics/apriltag-generation).

#### Get printable AprilTags

This is a helper function for generating a scalable SVG version of an AprilTag from a PNG image.

In [38]:
import os
import requests
from IPython.display import SVG, display

def generate_apriltag_svg(tag_name: str, tag_family: str = "tagStandard52h13", size_mm: int = 50):
  
    filename = f"{tag_name}.png"
    svg_filename = f"{tag_name}_{size_mm}mm.svg"
    png_url = f"https://raw.githubusercontent.com/AprilRobotics/apriltag-imgs/master/{tag_family}/{filename}"

    if not os.path.exists(filename):
        print(f"Downloading tag image from {png_url}...")
        r = requests.get(png_url)
        if r.status_code == 200:
            with open(filename, "wb") as f:
                f.write(r.content)
        else:
            raise ValueError(f"Failed to download image. Status code: {r.status_code}")

    if not os.path.exists("tag_to_svg.py"):
        print("Downloading tag_to_svg.py...")
        script_url = "https://raw.githubusercontent.com/AprilRobotics/apriltag-imgs/master/tag_to_svg.py"
        r = requests.get(script_url)
        with open("tag_to_svg.py", "wb") as f:
            f.write(r.content)

    print(f"Converting to SVG with size {size_mm}mm...")
    os.system(f"python tag_to_svg.py {filename} {svg_filename} --size={size_mm}mm")

    if os.path.exists(svg_filename):
        display(SVG(filename=svg_filename))
    else:
        raise FileNotFoundError(f"{svg_filename} not generated.")



To generate a printable SVG of an AprilTag, use the function below by specifying the full tag name (e.g., "tag41_12_00110") and its tag family (e.g., "tagStandard41h12"). You can also adjust the size_mm parameter to control the output size (in millimeters).

In [50]:
generate_apriltag_svg("tag41_12_00110", tag_family="tagStandard41h12", size_mm=60)

Converting to SVG with size 60mm...
